In [75]:
import pandas as pd
import numpy as np
import collections
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model

In [51]:
#Lendo conjuntos de teste e treino
#train_validation = pd.read_csv('train.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [37]:
def train_validate_split(df, train_percent=.7, validate_percent=.3, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    return train, validate

In [38]:
#Divide o dataset de treino entre treino e validação (70% e 30%)
train, validate = train_validate_split(train_validation)

In [39]:
print("Full train:" + str(len(train_validation)) + ", train:" + str(len(test)) + ", validate:" + str(len(validate)))

Full train:595212, train:892816, validate:178563


In [52]:
#Dez primeiras linhas do conjunto de treino
train.head(10)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0
5,19,0,5,1,4,0,0,0,0,0,...,4,2,0,9,0,1,0,1,1,1
6,20,0,2,1,3,1,0,0,1,0,...,3,0,0,10,0,1,0,0,1,0
7,22,0,5,1,4,0,0,1,0,0,...,7,1,3,6,1,0,1,0,1,0
8,26,0,5,1,3,1,0,0,0,1,...,4,2,1,5,0,1,0,0,0,1
9,28,1,1,1,2,0,0,0,1,0,...,3,5,0,6,0,1,0,0,1,0


In [41]:
#Dez primeiras linhas do conjunto de treino
test.head(10)

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,...,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,...,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,...,4,0,0,4,0,1,1,0,0,1
5,5,0,1,6,0,0,1,0,0,0,...,8,1,4,9,1,0,1,0,1,0
6,6,0,1,3,0,0,0,1,0,0,...,2,0,4,6,1,1,0,0,0,0
7,8,0,1,0,0,0,1,0,0,0,...,3,1,4,9,0,1,0,0,0,0
8,10,0,1,7,0,0,0,1,0,0,...,5,1,4,6,0,0,1,0,0,0
9,11,1,1,6,0,0,0,0,0,1,...,6,1,6,10,0,1,1,0,0,0


In [54]:
#Mostrando a lista dos headers
list(test)

['id',
 'ps_ind_01',
 'ps_ind_02_cat',
 'ps_ind_03',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_10_bin',
 'ps_ind_11_bin',
 'ps_ind_12_bin',
 'ps_ind_13_bin',
 'ps_ind_14',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_14',
 'ps_car_15',
 'ps_calc_01',
 'ps_calc_02',
 'ps_calc_03',
 'ps_calc_04',
 'ps_calc_05',
 'ps_calc_06',
 'ps_calc_07',
 'ps_calc_08',
 'ps_calc_09',
 'ps_calc_10',
 'ps_calc_11',
 'ps_calc_12',
 'ps_calc_13',
 'ps_calc_14',
 'ps_calc_15_bin',
 'ps_calc_16_bin',
 'ps_calc_17_bin',
 'ps_calc_18_bin',
 'ps_calc_19_bin',
 'ps_calc_20_bin']

In [12]:
#Criação de novo DataFrame a partir de One-Hot encoding
new_data_train = pd.get_dummies(
    data=train, 
    prefix=['ps_ind_02_cat_encoded', 'ps_ind_04_cat_encoded',
            'ps_ind_05_cat_encoded', 'ps_car_01_cat_encoded',
            'ps_car_02_cat_encoded', 'ps_car_03_cat_encoded',
            'ps_car_04_cat_encoded', 'ps_car_05_cat_encoded',
            'ps_car_06_cat_encoded', 'ps_car_07_cat_encoded',
            'ps_car_08_cat_encoded', 'ps_car_09_cat_encoded',
            'ps_car_10_cat_encoded', 'ps_car_11_cat_encoded'], 
    columns=['ps_ind_02_cat', 'ps_ind_04_cat',
            'ps_ind_05_cat', 'ps_car_01_cat',
            'ps_car_02_cat', 'ps_car_03_cat',
            'ps_car_04_cat', 'ps_car_05_cat',
            'ps_car_06_cat', 'ps_car_07_cat',
            'ps_car_08_cat', 'ps_car_09_cat',
            'ps_car_10_cat', 'ps_car_11_cat'])

#Criação de novo DataFrame a partir de One-Hot encoding
new_data_test = pd.get_dummies(
    data=test, 
    prefix=['ps_ind_02_cat_encoded', 'ps_ind_04_cat_encoded',
            'ps_ind_05_cat_encoded', 'ps_car_01_cat_encoded',
            'ps_car_02_cat_encoded', 'ps_car_03_cat_encoded',
            'ps_car_04_cat_encoded', 'ps_car_05_cat_encoded',
            'ps_car_06_cat_encoded', 'ps_car_07_cat_encoded',
            'ps_car_08_cat_encoded', 'ps_car_09_cat_encoded',
            'ps_car_10_cat_encoded', 'ps_car_11_cat_encoded'], 
    columns=['ps_ind_02_cat', 'ps_ind_04_cat',
            'ps_ind_05_cat', 'ps_car_01_cat',
            'ps_car_02_cat', 'ps_car_03_cat',
            'ps_car_04_cat', 'ps_car_05_cat',
            'ps_car_06_cat', 'ps_car_07_cat',
            'ps_car_08_cat', 'ps_car_09_cat',
            'ps_car_10_cat', 'ps_car_11_cat'])

new_data_train.head(10)

,id,target,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,...,ps_car_11_cat_encoded_95,ps_car_11_cat_encoded_96,ps_car_11_cat_encoded_97,ps_car_11_cat_encoded_98,ps_car_11_cat_encoded_99,ps_car_11_cat_encoded_100,ps_car_11_cat_encoded_101,ps_car_11_cat_encoded_102,ps_car_11_cat_encoded_103,ps_car_11_cat_encoded_104
0,7,0,2,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,1,7,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,13,0,5,9,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16,0,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,17,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,19,0,5,4,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
6,20,0,2,3,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7,22,0,5,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,26,0,5,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,28,1,1,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [55]:
#Proporção de target 1 e 0
#new_data_train['target'].value_counts()
train['target'].value_counts()

0    573518
1     21694
Name: target, dtype: int64

In [56]:
#Separando features e target para criação do modelo
#x = new_data_train.drop('target', axis=1)
#y = new_data_train['target']
x = train.drop('target', axis=1)
y = train['target']

In [57]:
#Features não contém o campo target
x.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,2,2,5,1,0,0,1,0,0,...,9,1,5,8,0,1,1,0,0,1
1,9,1,1,7,0,0,0,0,1,0,...,3,1,1,9,0,1,1,0,1,0
2,13,5,4,9,1,0,0,0,1,0,...,4,2,7,7,0,1,1,0,1,0
3,16,0,1,2,0,0,1,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,2,0,1,0,1,0,0,0,...,3,1,1,3,0,0,0,1,1,0


In [58]:
#Features não contém o campo target
y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [76]:
#Criação do modelo e treino
#clf = DecisionTreeClassifier(max_depth=30) score 0.99260599584685794
#clf = linear_model.SGDRegressor() score #-1.806326181381535e+42
clf.fit(x, y)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [77]:
#Verificando score no conjunto de treino
clf.score(x, y)

-1.806326181381535e+42

In [48]:
#Verificando score no conjunto de validação
x_validate = validate.drop('target', axis=1)
y_validate = validate['target']
clf.score(x_validate, y_validate)

0.96257343346605961

In [78]:
#Array de resposta
collections.Counter(clf.predict(test))

Counter({-480473590040087.75: 1,
         -980941682343625.88: 1,
         -1013984123083013.6: 1,
         -1256746404849199.5: 1,
         -1885802703259975.3: 1,
         -1840277018904796.5: 1,
         -1977132697616605.8: 1,
         -3067062371801014.5: 1,
         -3593714203665937.0: 1,
         -3931514681251734.0: 1,
         -4233091214291391.5: 1,
         -4804159855668624.0: 1,
         -5115708339087149.0: 1,
         -5669903132240059.0: 1,
         -6852819846307132.0: 1,
         -7427195291793716.0: 1,
         -7584308212916373.0: 1,
         -8055840956835178.0: 1,
         -8282695199651282.0: 1,
         -9025561468289564.0: 1,
         -9301857383477604.0: 1,
         -9636678857470128.0: 1,
         -10156755150883442.0: 1,
         -9904129656121748.0: 1,
         -11156018239758338.0: 1,
         -11696441955777542.0: 1,
         -11954643825410178.0: 1,
         -12215621772435400.0: 1,
         -12358634939330374.0: 1,
         -12618260413310348.0: 1,
   

In [73]:
#Você precisa enviar ao Kaggle um arquivo .csv com as suas previsões. 
#Esse arquivo deve conter duas colunas com cabeçalho: id e target.
submission = pd.DataFrame()
submission['id'] = test['id']
submission['target'] = clf.predict(test)
submission.to_csv('submission.csv', index=False)